In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features

/content/drive/My Drive/SA/Code/spoof_detection_deep_features


In [2]:
!pip install librosa

     |████████████████████████████████| 1.6MB 8.3kB/s eta 0:00:02
     |████████████████████████████████| 327kB 143kB/s eta 0:00:01
     |████████████████████████████████| 399kB 15kB/s eta 0:00:018
     |████████████████████████████████| 112kB 10kB/s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.7.2-cp36-none-any.whl size=1612885 sha256=bcc9c9dbc31843a66e6773a002f60e863879649c53321be13a0ccbbae83e5975
  Stored in directory: /home/rohit/.cache/pip/wheels/4c/6e/d7/bb93911540d2d1e44d690a1561871e5b6af82b69e80938abef
  Created wheel for audioread: filename=audioread-2.1.8-cp36-none-any.whl size=23092 sha256=b1f1479d7d13ac78582461e1cdb9c67bc130d3f533b6d2398f317f9cfd028f4b
  Stored in directory: /home/rohit/.cache/pip/wheels/b9/64/09/0b6417df9d8ba8bc61a7d2553c5cebd714ec169644c88fc012
  Created wheel for resampy: filename=resampy-0.2.2-cp36-none-any.whl size=320719 sha256=5a0b7db2435f29f19c556809e259f403d5137db31b1826e77cb707cf9077160f
  Stored in directory: /home/rohit/.cache/pip

In [3]:
import keras 
import numpy as np
import librosa as lb
import pandas as pd
import os
import sys
# import pandas as pd
from keras.utils import to_categorical
# import soundfile as sf
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model,to_categorical
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
# import speechpy as sp
# import statistics
from keras import backend as K
from keras.layers import Dense, Activation, Flatten

In [12]:
filename = "/media/rohit/New Volume/codes/SA/spoof_detection_deep_features/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt"

# open the file for reading
filehandle = open(filename, 'r')
train_protocol = []
while True:
    # read a single line
    line = (filehandle.readline())
    train_protocol.append(line)
    if not line:
        break

# close the pointer to that file
filehandle.close()

In [13]:
train_protocol = [s[:-1] for s in train_protocol]

In [14]:
train_protocol = pd.DataFrame([s.split(' ') for s in train_protocol])

In [15]:
train_protocol.head()

,0,1,2,3,4
0,LA_0079,LA_T_1138215,-,-,bonafide
1,LA_0079,LA_T_1271820,-,-,bonafide
2,LA_0079,LA_T_1272637,-,-,bonafide
3,LA_0079,LA_T_1276960,-,-,bonafide
4,LA_0079,LA_T_1341447,-,-,bonafide


In [16]:
train_protocol.columns = ['speaker_id','file_id', 'blah','system_id', 'label']

In [17]:
train_protocol = train_protocol[['speaker_id', 'file_id', 'system_id', 'label']]

In [18]:
train_protocol

,speaker_id,file_id,system_id,label
0,LA_0079,LA_T_1138215,-,bonafide
1,LA_0079,LA_T_1271820,-,bonafide
2,LA_0079,LA_T_1272637,-,bonafide
3,LA_0079,LA_T_1276960,-,bonafide
4,LA_0079,LA_T_1341447,-,bonafide
5,LA_0079,LA_T_1363611,-,bonafide
6,LA_0079,LA_T_1596451,-,bonafide
7,LA_0079,LA_T_1608170,-,bonafide
8,LA_0079,LA_T_1684951,-,bonafide
9,LA_0079,LA_T_1699801,-,bonafide


In [19]:
train_protocol = train_protocol.dropna()

In [22]:
#import names of files in dataset
path = r'/media/rohit/New Volume/codes/SA/spoof_detection_deep_features/ASVspoof2019/LA/ASVspoof2019_LA_train/flac'
files = []
missing=[]
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.flac' in file  :        
            files.append(os.path.join(r, file))
        else:
            missing.append(file)
print(len(files))


/media/rohit/New Volume/codes/SA/spoof_detection_deep_features/ASVspoof2019/LA/ASVspoof2019_LA_train/flac
25380


In [23]:
files = [s.split('/') for s in files]

In [24]:
files = [s[-1] for s in files]

In [25]:
files = [s[:-5] for s in files]

In [26]:
files1 = [s.split(' ') for s in files]

In [27]:
for s in files:
    if len(s)>12:
        print(s)
        files.remove(s)

In [28]:
train_file_id = list(train_protocol.iloc[:,1])

In [29]:
train_file_id

['LA_T_1138215',
 'LA_T_1271820',
 'LA_T_1272637',
 'LA_T_1276960',
 'LA_T_1341447',
 'LA_T_1363611',
 'LA_T_1596451',
 'LA_T_1608170',
 'LA_T_1684951',
 'LA_T_1699801',
 'LA_T_1703395',
 'LA_T_1736342',
 'LA_T_1779188',
 'LA_T_1786825',
 'LA_T_1787246',
 'LA_T_1905558',
 'LA_T_2205687',
 'LA_T_2220901',
 'LA_T_2361751',
 'LA_T_2373806',
 'LA_T_2417641',
 'LA_T_2520083',
 'LA_T_2562689',
 'LA_T_2564579',
 'LA_T_2732709',
 'LA_T_2759900',
 'LA_T_2838981',
 'LA_T_2873890',
 'LA_T_2938316',
 'LA_T_3131990',
 'LA_T_3141223',
 'LA_T_3167734',
 'LA_T_3187715',
 'LA_T_3215578',
 'LA_T_3247389',
 'LA_T_3346004',
 'LA_T_3441957',
 'LA_T_3561136',
 'LA_T_3576775',
 'LA_T_3580508',
 'LA_T_3635733',
 'LA_T_3636245',
 'LA_T_3810183',
 'LA_T_3849267',
 'LA_T_3995089',
 'LA_T_3999087',
 'LA_T_3999267',
 'LA_T_4053835',
 'LA_T_4118798',
 'LA_T_4155454',
 'LA_T_4179989',
 'LA_T_4197307',
 'LA_T_4237787',
 'LA_T_4239283',
 'LA_T_4244329',
 'LA_T_4339311',
 'LA_T_4362109',
 'LA_T_4428691',
 'LA_T_4752788

In [34]:
files

['LA_T_5569255',
 'LA_T_1000137',
 'LA_T_1000406',
 'LA_T_1000648',
 'LA_T_1000824',
 'LA_T_1001074',
 'LA_T_1001114',
 'LA_T_1001169',
 'LA_T_1001718',
 'LA_T_1001871',
 'LA_T_1002656',
 'LA_T_1003665',
 'LA_T_1004407',
 'LA_T_1004631',
 'LA_T_1004644',
 'LA_T_1005349',
 'LA_T_1006715',
 'LA_T_1006969',
 'LA_T_1007217',
 'LA_T_5569411',
 'LA_T_5569559',
 'LA_T_5569977',
 'LA_T_5570263',
 'LA_T_5570559',
 'LA_T_5570596',
 'LA_T_5570855',
 'LA_T_5571064',
 'LA_T_5571139',
 'LA_T_5571161',
 'LA_T_5571256',
 'LA_T_5571720',
 'LA_T_5571723',
 'LA_T_5572362',
 'LA_T_5572375',
 'LA_T_5572610',
 'LA_T_5573853',
 'LA_T_5573999',
 'LA_T_5574101',
 'LA_T_5574751',
 'LA_T_5574824',
 'LA_T_3410262',
 'LA_T_3410399',
 'LA_T_3410679',
 'LA_T_3411011',
 'LA_T_3411067',
 'LA_T_3411293',
 'LA_T_3412575',
 'LA_T_3413283',
 'LA_T_3413393',
 'LA_T_3413479',
 'LA_T_3413823',
 'LA_T_3414698',
 'LA_T_3415095',
 'LA_T_3415113',
 'LA_T_3415269',
 'LA_T_3415457',
 'LA_T_3415920',
 'LA_T_3416553',
 'LA_T_3416785

In [0]:
train_labels = []
train_audio = []
for count,audio in enumerate(files):
    index = train_file_id.index(audio)
    if bool(index) == True:
        train_audio.append(lb.load('/content/drive/My Drive/SA/Code/spoof_detection_deep_features/data/ASVspoof2019/LA/ASVspoof2019_LA_train/flac/'+audio+'.flac',sr=16000))
        train_labels.append(train_protocol.iloc[index,3])
    if count%100 == 0 :
        print(count)
np.save("",np.array(train_audio))
np.save("",train_labels)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [42]:
train_protocol_ordered_based_on_files = pd.DataFrame(columns=['speaker_id','file_id','system_id','label'])
train_protocol_ordered_based_on_files

,speaker_id,file_id,system_id,label


In [43]:
train_protocol_ordered_based_on_files = pd.DataFrame(columns=['speaker_id','file_id','system_id','label'])
for count,audio in enumerate(files):
    index = train_file_id.index(audio)
    if bool(index) == True:
        train_protocol_ordered_based_on_files=train_protocol_ordered_based_on_files.append(train_protocol.iloc[index])
    if count%100 == 0 :
        print(count)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [44]:
train_protocol_ordered_based_on_files

,speaker_id,file_id,system_id,label
19209,LA_0086,LA_T_5569255,A05,spoof
16820,LA_0094,LA_T_1000137,A04,spoof
1280,LA_0089,LA_T_1000406,-,bonafide
6194,LA_0098,LA_T_1000648,A01,spoof
16624,LA_0093,LA_T_1000824,A04,spoof
10366,LA_0080,LA_T_1001074,A03,spoof
18152,LA_0081,LA_T_1001114,A05,spoof
4656,LA_0090,LA_T_1001169,A01,spoof
6195,LA_0098,LA_T_1001718,A01,spoof
17016,LA_0095,LA_T_1001871,A04,spoof


In [47]:
train_protocol_ordered_based_on_files.to_pickle("/media/rohit/New Volume/codes/SA/spoof_detection_deep_features/train_protocol_ordered_based_on_files.pkl")

In [49]:
unique, counts = np.unique(train_protocol_ordered_based_on_files.iloc[:,3], return_counts=True)
print(dict(zip(unique, counts)))

{'bonafide': 2579, 'spoof': 22800}


In [30]:
index = train_file_id.index('LA_T_4941896')
print(train_protocol.iloc[index])

speaker_id         LA_0087
file_id       LA_T_4941896
system_id              A01
label                spoof
Name: 4170, dtype: object


In [3]:
import numpy as np
import pandas as pd
import pickle

audio = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train.npy")
labels = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_labels.npy")
train_protocol_ordered_based_on_files = pickle.load(open('/home/rohita/rohit/spoof/npy_data_asvspoof/train_protocol_ordered_based_on_files.pkl','rb'))

unique, counts = np.unique(train_protocol_ordered_based_on_files.iloc[:,3], return_counts=True)
print(dict(zip(unique, counts)))

dataset = pd.DataFrame(audio)
dataset = dataset.join(train_protocol_ordered_based_on_files)


shuffled_df = dataset.sample(frac=1,random_state=4)

# Put all the human class in a separate dataset.
human_samples = shuffled_df.loc[shuffled_df['label'] == 1]

#Randomly select 3674 observations from the spoofed (majority class)
spoofed_samples = shuffled_df.loc[shuffled_df['label'] == 0].sample(n=2579,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([human_samples, spoofed_samples])

ValueError: unsupported pickle protocol: 4

In [0]:
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)